In [18]:
##data inges

from snowflake.snowpark import Session

# Assuming `session` is already created and connected via your GitHub setup
# If you're using Snowpark, the session object should already be available.
# If not, you'd create it using the following:
# session = Session.builder.configs(connection_options).create()

# Create a stage (if not already created)


# Create the table
create_table_query = """
CREATE OR REPLACE TABLE RAW_CO2.Daily_Measurements (
    date STRING,
    co2_ppm FLOAT
);
"""
session.sql(create_table_query).collect()

# Copy data from the stage into the table
copy_query = """
COPY INTO RAW_CO2.Daily_Measurements
    FROM @RAW_CO2.CO2_EXTERNAL_STAGE
    FILE_FORMAT = (
        TYPE = 'CSV' 
        SKIP_HEADER = 1
        FIELD_OPTIONALLY_ENCLOSED_BY = '"'
    )
    ON_ERROR = 'CONTINUE';
"""
session.sql(copy_query).collect()

# Create a stream on the table
create_stream_query = """
CREATE OR REPLACE STREAM RAW_CO2.DAILY_MEASUREMENTS_STREAM 
ON TABLE RAW_CO2.Daily_Measurements;
"""
session.sql(create_stream_query).collect()

# Optionally, verify if the table and stream are created successfully
print("Stage, table, and stream are created successfully.")

# Close session if needed (depends on whether you need it in the current script)
session.close()


SyntaxError: invalid syntax (1166821928.py, line 1)